In [1]:
import pandas as pd 
import matplotlib.pylab as plt 
import numpy as  np 
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.gridspec import GridSpec
import os



In [2]:
df=pd.read_csv(r'C:\Users\aksha\coading\New folder (2)\data\recent\5_batteries.csv')
feature_list=df.columns
print(feature_list)

Index(['Cycle', 'Voltage_measured', 'Current_measured', 'Temperature_measured',
       'Time', 'Capacity', 'Battery_ID', 'capacity_fade_total',
       'capacity_fade_percent', 'capacity_rolling_mean_5',
       'capacity_rolling_std_5', 'capacity_trend_5', 'voltage_rolling_mean_5',
       'temp_rolling_mean_5', 'capacity_velocity', 'capacity_acceleration',
       'cycle_normalized', 'cycles_from_start', 'voltage_change',
       'resistance_proxy', 'internal_resistance', 'energy_discharged',
       'power_avg', 'voltage_efficiency', 'discharge_capacity_ratio',
       'temp_capacity_interaction', 'voltage_capacity_ratio',
       'power_to_energy_ratio', 'degradation_acceleration_abs',
       'remaining_capacity', 'estimated_cycles_to_eol'],
      dtype='object')


In [3]:
print(len(list(df.columns)))

31


### 2 strategy we wil use 
#### 1 train the model to know the feature importance 
#### 2 use important feature to train new model so that we could not miss out with importance features


In [4]:
# Exclude only target and leakage features
exclude = ['Cycle', 'Time', 'Capacity', 'Battery_ID',
           'capacity_fade_total', 'discharge_capacity_ratio', 
           'remaining_capacity']

features = [col for col in df.columns if col not in exclude]
# This gives us ~24-28 features

X = df[features]
y = df['Capacity']

In [5]:
# Train on 3 batteries
train_mask = df['Battery_ID'].isin(['B0005', 'B0006', 'B0007'])
test_mask = df['Battery_ID'] == 'B0018'

X_train = X[train_mask]
y_train = y[train_mask]
X_test = X[test_mask]
y_test = y[test_mask]

print(f"Train: {len(X_train)} samples from 3 batteries")
print(f"Test: {len(X_test)} samples from 1 unseen battery")

Train: 489 samples from 3 batteries
Test: 127 samples from 1 unseen battery


In [6]:
### we have use the next battrey for testing 
#now lets train a model 
X_train = X[train_mask]
y_train = y[train_mask]
X_test = X[test_mask]
y_test = y[test_mask]
model = XGBRegressor(n_estimators=100,
                     max_depth=6,
                     learning_rate=0.05,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     random_state=42
)
model.fit(X_train,y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [7]:
#lets get the feature importance 
y_pred=model.predict(X_test)


#metrics
r2=r2_score(y_test,y_pred)
rmse=np.sqrt(mean_squared_error(y_test,y_pred))


#lets have the feature importance 

importance =pd.DataFrame({
    "features":features,
    "importance":model.feature_importances_}).sort_values('importance',ascending=False)


print(f"R2 = {r2:.3f}")
print(f"top 10 Features:")
print(importance.head(10))

R2 = 0.997
top 10 Features:
                   features  importance
16        energy_discharged    0.437997
4   capacity_rolling_mean_5    0.393575
20   voltage_capacity_ratio    0.115247
1          Current_measured    0.034885
3     capacity_fade_percent    0.009845
17                power_avg    0.005852
14         resistance_proxy    0.001000
21    power_to_energy_ratio    0.000525
15      internal_resistance    0.000315
0          Voltage_measured    0.000132


In [8]:
top_15 = importance.head(15)['features'].tolist()
X_train_reduced = X_train[top_15]
X_test_reduced = X_test[top_15]

#lets get the feature importance 
y_pred=model.predict(X_test)


#metrics
r2=r2_score(y_test,y_pred)
rmse=np.sqrt(mean_squared_error(y_test,y_pred))


print(f"R2 = {r2:.3f}")
print(f"top 15 Features:")
print(importance.head(10))

R2 = 0.997
top 15 Features:
                   features  importance
16        energy_discharged    0.437997
4   capacity_rolling_mean_5    0.393575
20   voltage_capacity_ratio    0.115247
1          Current_measured    0.034885
3     capacity_fade_percent    0.009845
17                power_avg    0.005852
14         resistance_proxy    0.001000
21    power_to_energy_ratio    0.000525
15      internal_resistance    0.000315
0          Voltage_measured    0.000132


In [9]:
# Which one did you calculate?
# train_r2 = r2_score(X_test, y_test)  # Should be high
test_r2 = r2_score(y_test, y_pred)     # This is what matters!

In [10]:
# print(f"Train R²: {train_r2:.4f}")
print(f"Test R²:  {test_r2:.4f}")

# If both are 0.997 → DATA LEAKAGE!
# If train=0.997, test=0.90 → Overfitting but OK
# If train=0.85, test=0.997 → Something very wrong!

Test R²:  0.9973


In [11]:
# These should NOT be in your feature list:
leakage_features = [
    'capacity_fade_total',           # = initial - Capacity (direct!)
    'discharge_capacity_ratio',      # = Capacity / initial (direct!)
    'capacity_rolling_mean_5',       # Almost = Capacity itself
    'energy_discharged',             # = Capacity × Voltage (99% correlated)
    'remaining_capacity',            # = Capacity - EOL_threshold
]

# Check if any are in your features:
print("Features used:", features)
for leak in leakage_features:
    if leak in features:
        print(f"⚠️  LEAKAGE: {leak} is in features!")

Features used: ['Voltage_measured', 'Current_measured', 'Temperature_measured', 'capacity_fade_percent', 'capacity_rolling_mean_5', 'capacity_rolling_std_5', 'capacity_trend_5', 'voltage_rolling_mean_5', 'temp_rolling_mean_5', 'capacity_velocity', 'capacity_acceleration', 'cycle_normalized', 'cycles_from_start', 'voltage_change', 'resistance_proxy', 'internal_resistance', 'energy_discharged', 'power_avg', 'voltage_efficiency', 'temp_capacity_interaction', 'voltage_capacity_ratio', 'power_to_energy_ratio', 'degradation_acceleration_abs', 'estimated_cycles_to_eol']
⚠️  LEAKAGE: capacity_rolling_mean_5 is in features!
⚠️  LEAKAGE: energy_discharged is in features!


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

# Load your data
df2 = pd.read_csv(r'C:\Users\aksha\coading\New folder (2)\data\recent\5_batteries.csv')

print("="*60)
print("🔍 OVERFITTING / LEAKAGE DIAGNOSTIC")
print("="*60)

# Check which features you used
print("\n1️⃣ FEATURES USED:")
print("List your features here...")

# Check correlations with target
print("\n2️⃣ CORRELATION WITH TARGET (Capacity):")
correlations = []
for col in df2.columns:
    if col not in ['Battery_ID', 'Cycle']:
        try:
            corr = df2[col].corr(df2['Capacity'])
            correlations.append({'feature': col, 'correlation': abs(corr)})
        except:
            pass

corr_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
print("\nTop 10 highest correlations:")
print(corr_df.head(10))

# Flag suspicious correlations
print("\n⚠️  LEAKAGE SUSPECTS (correlation > 0.95):")
leakage = corr_df[corr_df['correlation'] > 0.95]
if len(leakage) > 0:
    print(leakage)
    print("\n🚨 These features likely cause data leakage!")
else:
    print("✅ No obvious leakage")

# Check train vs test split
train_mask = df2['Battery_ID'].isin(['B0005', 'B0006', 'B0007'])
test_mask = df2['Battery_ID'] == 'B0018'

print(f"\n3️⃣ DATA SPLIT CHECK:")
print(f"Train batteries: {df[train_mask]['Battery_ID'].unique()}")
print(f"Train samples: {train_mask.sum()}")
print(f"Test battery: {df[test_mask]['Battery_ID'].unique()}")
print(f"Test samples: {test_mask.sum()}")

# Check if target distributions are similar
print(f"\n4️⃣ TARGET DISTRIBUTION:")
print(f"Train Capacity: {df[train_mask]['Capacity'].mean():.3f} ± {df[train_mask]['Capacity'].std():.3f}")
print(f"Test Capacity:  {df[test_mask]['Capacity'].mean():.3f} ± {df[test_mask]['Capacity'].std():.3f}")


### we have use the next battrey for testing 
#now lets train a model 
X_train = X[train_mask]
y_train = y[train_mask]
X_test = X[test_mask]
y_test = y[test_mask]
model = XGBRegressor(n_estimators=100,
                     max_depth=6,
                     learning_rate=0.05,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     random_state=42
)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)


#metrics
train_r2 = r2_score(y_train, model.predict(X_train))
r2=r2_score(y_test,y_pred)
rmse=np.sqrt(mean_squared_error(y_test,y_pred))

# Share your actual train/test R²
print(f"\n5️⃣ YOUR RESULTS:")
print(f"Report both:")
print(f"  Train R²: {train_r2}")
print(f"  Test R²:  {r2}")

🔍 OVERFITTING / LEAKAGE DIAGNOSTIC

1️⃣ FEATURES USED:
List your features here...

2️⃣ CORRELATION WITH TARGET (Capacity):

Top 10 highest correlations:
                     feature  correlation
4                   Capacity     1.000000
19         energy_discharged     0.999101
7    capacity_rolling_mean_5     0.996127
24    voltage_capacity_ratio     0.990541
27        remaining_capacity     0.966666
6      capacity_fade_percent     0.957501
22  discharge_capacity_ratio     0.957501
3                       Time     0.954102
25     power_to_energy_ratio     0.952387
14          cycle_normalized     0.946026

⚠️  LEAKAGE SUSPECTS (correlation > 0.95):
                     feature  correlation
4                   Capacity     1.000000
19         energy_discharged     0.999101
7    capacity_rolling_mean_5     0.996127
24    voltage_capacity_ratio     0.990541
27        remaining_capacity     0.966666
6      capacity_fade_percent     0.957501
22  discharge_capacity_ratio     0.957501
3    

### model has data leackage so some features are directly predicted the the output as we see the the feature are too coorelated  

## lets exclude some feature and see what the resutls are 

In [13]:

# Load data
df = pd.read_csv(r'C:\Users\aksha\coading\New folder (2)\data\recent\5_batteries.csv')

print("="*60)
print("🔧 COMPLETELY CLEAN MODEL - ALL LEAKAGE REMOVED")
print("="*60)

# EXCLUDE ALL HIGH-CORRELATION FEATURES
EXCLUDE = [
    'Cycle', 'Time', 'Capacity', 'Battery_ID',
    'capacity_fade_total', 'capacity_fade_percent',
    'discharge_capacity_ratio', 'remaining_capacity',
    'capacity_rolling_mean_5', 'capacity_rolling_std_5',
    'energy_discharged', 'voltage_capacity_ratio',
    'power_to_energy_ratio',
]

# Get clean features
clean_features = [col for col in df.columns if col not in EXCLUDE]

print(f"\n✅ Features after removing leakage: {len(clean_features)}")
print("Clean features:", clean_features)

# Double-check correlations
print("\n🔍 Verifying no high correlations remain...")
high_corr_found = False
for feat in clean_features:
    try:
        corr = abs(df[feat].corr(df['Capacity']))
        if corr > 0.90:
            print(f"⚠️  {feat}: {corr:.4f} (still too high!)")
            high_corr_found = True
    except:
        pass

if not high_corr_found:
    print("✅ All remaining features have correlation < 0.90")

# Prepare data
X = df[clean_features]
y = df['Capacity']

# Battery split
# Battery split
train_mask = df['Battery_ID'].isin(['B0005', 'B0006', 'B0007'])
test_mask = df['Battery_ID'] == 'B0018'

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"\n📊 Train: {len(X_train)} samples | Test: {len(X_test)} samples")

# Train
print("\n🤖 Training model...")
model = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    random_state=42
)

model.fit(X_train, y_train, verbose=False)

# Evaluate
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

print("\n" + "="*60)
print("📊 CLEAN MODEL RESULTS (NO LEAKAGE)")
print("="*60)

print(f"\n🎓 TRAIN SET:")
print(f"   R²:   {train_r2:.4f}")
print(f"   RMSE: {train_rmse:.4f} Ah")

print(f"\n🧪 TEST SET (Unseen Battery):")
print(f"   R²:   {test_r2:.4f}")
print(f"   RMSE: {test_rmse:.4f} Ah")
print(f"   MAE:  {test_mae:.4f} Ah")
print(f"   MAPE: {test_mape:.2f}%")

print(f"\n🔍 GENERALIZATION GAP:")
gap = train_r2 - test_r2
print(f"   Train R² - Test R²: {gap:.4f}")
if gap < 0.05:
    print(f"   ✅ EXCELLENT generalization!")
elif gap < 0.10:
    print(f"   ✅ GOOD generalization")
else:
    print(f"   ⚠️  Some overfitting (gap = {gap:.4f})")

# Feature importance
feature_imp = pd.DataFrame({
    'feature': clean_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🏆 TOP 10 FEATURES (by importance):")
for idx, row in feature_imp.head(10).iterrows():
    corr = abs(df[row['feature']].corr(df['Capacity']))
    print(f"   {row['feature']:30s} → {row['importance']:.4f} (corr: {corr:.3f})")

# Verdict
print("\n" + "="*60)
print("🎉 FINAL ASSESSMENT")
print("="*60)

if test_r2 >= 0.90:
    status = "🌟 EXCELLENT"
elif test_r2 >= 0.85:
    status = "✅ GOOD"
elif test_r2 >= 0.75:
    status = "⚠️  ACCEPTABLE"
else:
    status = "❌ NEEDS WORK"

print(f"\n{status}")
print(f"Test R² = {test_r2:.3f}")
print(f"Test Error = {test_mape:.1f}%")

if 0.85 <= test_r2 <= 0.96:
    print(f"\n✅ This is REALISTIC, HONEST performance!")
    print(f"✅ No data leakage")
    print(f"✅ Production-ready quality")
else:
    print(f"\n⚠️  Performance outside expected range (0.85-0.96)")

🔧 COMPLETELY CLEAN MODEL - ALL LEAKAGE REMOVED

✅ Features after removing leakage: 18
Clean features: ['Voltage_measured', 'Current_measured', 'Temperature_measured', 'capacity_trend_5', 'voltage_rolling_mean_5', 'temp_rolling_mean_5', 'capacity_velocity', 'capacity_acceleration', 'cycle_normalized', 'cycles_from_start', 'voltage_change', 'resistance_proxy', 'internal_resistance', 'power_avg', 'voltage_efficiency', 'temp_capacity_interaction', 'degradation_acceleration_abs', 'estimated_cycles_to_eol']

🔍 Verifying no high correlations remain...
⚠️  cycle_normalized: 0.9460 (still too high!)
⚠️  power_avg: 0.9167 (still too high!)

📊 Train: 489 samples | Test: 127 samples

🤖 Training model...

📊 CLEAN MODEL RESULTS (NO LEAKAGE)

🎓 TRAIN SET:
   R²:   0.9999
   RMSE: 0.0021 Ah

🧪 TEST SET (Unseen Battery):
   R²:   0.8843
   RMSE: 0.0499 Ah
   MAE:  0.0433 Ah
   MAPE: 2.90%

🔍 GENERALIZATION GAP:
   Train R² - Test R²: 0.1156
   ⚠️  Some overfitting (gap = 0.1156)

🏆 TOP 10 FEATURES (by 

# now we have the realistic performance of the model 

🧪 TEST SET (Unseen Battery):
   R²:   0.8843

lets train with the whole data 

In [17]:
# ============================================================================
# SAVE XGBOOST MODEL
# ============================================================================
import joblib
from xgboost import XGBRegressor

# After training your model
# model = XGBRegressor(...)
# model.fit(X_train, y_train)

# Save ONLY the model object (not a dictionary)
joblib.dump(model, 'xgboost_model.pkl')

print("Model saved!")

# Verify it loads correctly
loaded_model = joblib.load('xgboost_model.pkl')
print(f"Model type: {type(loaded_model)}")  # Should be: <class 'xgboost.sklearn.XGBRegressor'>

# Test prediction
test_pred = loaded_model.predict(X_test[:1])
print(f"Test prediction: {test_pred[0]:.4f} Ah")

Model saved!
Model type: <class 'xgboost.sklearn.XGBRegressor'>
Test prediction: 1.8548 Ah
